In [1]:
# Import BrainGlobe

from bg_atlasapi import show_atlases
import numpy as np
show_atlases()

                                                                                  
                                                                                  
                                Brainglobe Atlases                                
╭──────────────────────────────────┬────────────┬───────────────┬────────────────╮
│ Name                             │ Downloaded │ Local version │ Latest version │
├──────────────────────────────────┼────────────┼───────────────┼────────────────┤
│ allen_human_500um                │     ✔      │      0.1      │      0.1       │
│ allen_mouse_25um                 │     ✔      │      1.2      │      1.2       │
│ mpin_zfish_1um                   │     ✔      │      1.0      │      1.0       │
│ example_mouse_100um              │    ---     │      ---      │      1.2       │
│ allen_mouse_10um                 │    ---     │      ---      │      1.2       │
│ allen_mouse_50um                 │    ---     │      ---      │      1.2       │
│ allen_mouse_100um                │    ---     │      ---      │      1.2       │
│ kim_mouse_10um                   │    ---     │      ---      │      1.0       │
│ kim_mouse_25um                   │    ---     │      ---      │      1.0       │
│ kim_mouse_50um                   │    ---     │      ---      │      1.0       │
│ kim_mouse_100um                  │    ---     │      ---      │      1.0       │
│ osten_mouse_10um                 │    ---     │      ---      │      1.1       │
│ osten_mouse_25um                 │    ---     │      ---      │      1.1       │
│ osten_mouse_50um                 │    ---     │      ---      │      1.1       │
│ osten_mouse_100um                │    ---     │      ---      │      1.1       │
│ allen_cord_20um                  │    ---     │      ---      │      1.0       │
│ azba_zfish_4um                   │    ---     │      ---      │      1.0       │
│ whs_sd_rat_39um                  │    ---     │      ---      │      1.0       │
╰──────────────────────────────────┴────────────┴───────────────┴────────────────╯

In [ ]:
# Download the atlas of your choice taken from the list above

from bg_atlasapi.bg_atlas import BrainGlobeAtlas
#atlas = BrainGlobeAtlas("allen_human_500um")
#atlas = BrainGlobeAtlas("mpin_zfish_1um")
atlas = BrainGlobeAtlas("allen_mouse_25um")
print(atlas.atlas_name)

In [ ]:
# Atlas info
print(atlas.metadata)

In [ ]:
# Its resolution
min(atlas.metadata['resolution'])

In [ ]:
# If you want to see it in 3D
# import napari
# viewer = napari.view_image(reference_image)

In [ ]:
# Starting PyImageJ, some of these dependencies may be autodiscovered via transitive dependencies, not sure

imagej_core_dep = 'net.imagej:imagej:2.3.0'
imagej_legacy_dep = 'net.imagej:imagej-legacy:0.38.1'
bdv_core_dep = 'sc.fiji:bigdataviewer-core:10.2.1'
imglib2_ui_dep = 'net.imglib2:imglib2-ui:2.0.1'
bdv_vistools_dep = 'sc.fiji:bigdataviewer-vistools:1.0.0-beta-29'
imglib2_cache_dep = 'net.imglib2:imglib2-cache:1.0.0-beta-16'
abba_dep = 'ch.epfl.biop:ImageToAtlasRegister:0.2.3'

deps_pack = [imagej_core_dep, imagej_legacy_dep, bdv_core_dep,\
             imglib2_ui_dep, bdv_vistools_dep,imglib2_cache_dep, abba_dep ];

In [ ]:
# Starts ImageJ, the UI will show up
import imagej
ij = imagej.init(deps_pack, headless=False)
ij.ui().showUI()

In [ ]:
# Importing necessary classes from Java for the next cell, the hard one

from scyjava import jimport

import jpype
import jpype.imports
from jpype.types import *
from jpype import JImplements, JOverride

BdvFunctions = jimport('bdv.util.BdvFunctions')
Atlas = jimport('ch.epfl.biop.atlas.struct.Atlas')
AtlasMap = jimport('ch.epfl.biop.atlas.struct.AtlasMap')
AtlasOntology = jimport('ch.epfl.biop.atlas.struct.AtlasOntology')
AtlasNode = jimport('ch.epfl.biop.atlas.struct.AtlasNode')
AtlasHelper = jimport('ch.epfl.biop.atlas.struct.AtlasHelper')
AffineTransform3D = jimport('net.imglib2.realtransform.AffineTransform3D')
ArrayList = jimport('java.util.ArrayList')
BdvOptions = jimport('bdv.util.BdvOptions')

In [ ]:
# The core of this compatibility layer : 
# - translates BrainGlobe API to ch.epfl.biop.atlas.struct interfaces through JPype

@JImplements(AtlasMap)
class BrainGlobeMap(object):
    
    def __init__(self, bg_atlas):
        #this function is called way too many times if I put here the content 
        # of initialize... and I don't know why
        # that's why there's this initialize function
        self.atlas = bg_atlas
    
    @JOverride
    def setDataSource(self, dataSource):
        self.dataSource = dataSource

    @JOverride
    def initialize(self, atlasName):
        self.atlasName = str(atlasName)
        
        atlas_resolution_in__mm = JDouble(min(self.atlas.metadata['resolution'])/1000.0)
        
        vox_x_mm = self.atlas.metadata['resolution'][0] / 1000.0
        vox_y_mm = self.atlas.metadata['resolution'][1] / 1000.0
        vox_z_mm = self.atlas.metadata['resolution'][2] / 1000.0
        
        affine_transform = AffineTransform3D()
        affine_transform.scale(JDouble(vox_x_mm), JDouble(vox_y_mm), JDouble(vox_z_mm))
        
        # Convert 
        bss = BdvFunctions.show(ij.py.to_java(self.atlas.reference),JString(self.atlas.atlas_name+'_reference'), BdvOptions.options().sourceTransform(affine_transform))
        reference_sac = bss.getSources().get(0)
        bss.getBdvHandle().close()
        
        bss = BdvFunctions.show(ij.py.to_java(self.atlas.hemispheres),JString(self.atlas.atlas_name+'_hemispheres'), BdvOptions.options().sourceTransform(affine_transform))
        left_right_sac = bss.getSources().get(0)
        bss.getBdvHandle().close()
        
        bss = BdvFunctions.show(ij.py.to_java(self.atlas.annotation),JString(self.atlas.atlas_name+'_annotation'), BdvOptions.options().sourceTransform(affine_transform))
        self.annotation_sac = bss.getSources().get(0)
        bss.getBdvHandle().close()
        
        image_keys = ArrayList()
        image_keys.add(JString('reference'))
        image_keys.add(JString('X'))
        image_keys.add(JString('Y'))
        image_keys.add(JString('Z'))
        image_keys.add(JString('Left Right'))
        
        structural_images = {
            'reference':  reference_sac,
            'X': AtlasHelper.getCoordinateSac(0,JString('X')),
            'Y': AtlasHelper.getCoordinateSac(1,JString('Y')),
            'Z': AtlasHelper.getCoordinateSac(2,JString('Z')),
            'Left Right': left_right_sac
        } #return Map<String,SourceAndConverter>
        
        self.atlas_resolution_in__mm = atlas_resolution_in__mm
        self.affine_transform = affine_transform
        self.image_keys = image_keys
        self.structural_images = structural_images
        self.maxReference = JDouble(np.max(atlas.reference)*2)

    @JOverride
    def getDataSource(self):
        return self.dataSource #return URL
        
    @JOverride
    def getStructuralImages(self):
        return self.structural_images
    
    @JOverride
    def getImagesKeys(self):
        return self.image_keys

    @JOverride
    def getLabelImage(self):
        return self.annotation_sac #SourceAndConverter

    @JOverride
    def getAtlasPrecisionInMillimeter(self):
        return self.atlas_resolution_in__mm

    @JOverride
    def getCoronalTransform(self):
        return AffineTransform3D()
    
    @JOverride
    def getImageMax(self, key):
         return self.maxReference #double
    
    @JOverride        
    def labelRight(self):
        return JInt(1)

    @JOverride
    def labelLeft(self):
        return JInt(2)
        
@JImplements(AtlasOntology)
class BrainGlobeOntology(object):
    
    def __init__(self, bg_atlas):
        self.atlas = bg_atlas
        
    @JOverride
    def getName(self):
        return JString(self.atlas.atlas_name)
    
    @JOverride
    def initialize(self):
        self.root_node = BrainGlobeAtlasNode(self.atlas, self.atlas.structures.tree.root, None)
        #self.labelToAtlasNodeMap = AtlasHelper.buildLabelToAtlasNodeMap(self.root_node)
        self.idToAtlasNodeMap = AtlasHelper.buildIdToAtlasNodeMap(self.root_node)

    @JOverride
    def setDataSource(self, dataSource):
        self.dataSource = dataSource

    @JOverride
    def getDataSource(self):
        return self.dataSource #return URL

    @JOverride
    def getRoot(self):  
        return self.root_node #return AtlasNode
    
    @JOverride
    def getNodeFromId(self, index):
        return self.idToAtlasNodeMap.get(index) #return AtlasNode 
    
    @JOverride
    def getNamingProperty(self):
        return self.namingProperty
    
    @JOverride
    def setNamingProperty(self, namingProperty):
        self.namingProperty = namingProperty
    
@JImplements(AtlasNode)
class BrainGlobeAtlasNode(object):

    def __init__(self, bg_atlas, index, parent_node):
        self.atlas = bg_atlas
        self.id = index
        self.parent_node = parent_node
        children_nodes = []
        for child in bg_atlas.structures.tree.children(index):
            childNode = BrainGlobeAtlasNode(atlas, child.identifier, self)
            children_nodes.append(childNode)
        self.children_nodes = ArrayList(children_nodes)
        self.namingKey = JString('acronym')
    
    @JOverride
    def getId(self):
        return JInt(self.id)
    
    @JOverride
    def getColor(self):
        val = JInt[3]
        rgb = self.data().get('rgb_triplet')
        return val

    @JOverride
    def data(self):
        return self.atlas.structures[self.id]
    
    @JOverride
    def parent(self):
        return self.parent_node

    @JOverride
    def children(self):
        return self.children_nodes
    
    @JOverride
    def toString(self):
        return self.data().get(self.namingKey)

    
@JImplements(Atlas)
class BrainGlobeAtlas(object):

    def __init__(self, bg_atlas):
        self.atlas = bg_atlas
        
    @JOverride
    def getMap(self):
        return self.bg_atlasmap
        
    @JOverride
    def getOntology(self):
        return self.bg_ontology
    
    @JOverride
    def initialize(self, mapURL, ontologyURL):
        self.bg_ontology = BrainGlobeOntology(self.atlas)
        self.bg_ontology.initialize()
        self.bg_ontology.setNamingProperty(JString('acronym'))
        self.bg_atlasmap = BrainGlobeMap(self.atlas)
        self.bg_atlasmap.initialize(self.atlas.atlas_name)
        self.dois = ArrayList()
        self.dois.add(JString('doi1')) #TODO
        self.dois.add(JString('doi2'))
    
    @JOverride
    def getDOIs(self):
        return self.dois

    @JOverride
    def getURL(self):
        return JString('BrainGlobe Atlas URL...')  
    
    @JOverride
    def getName(self):
        return JString(self.atlas.atlas_name)
    
    @JOverride
    def toString(self):
        return self.getName()

In [ ]:
# Makes the atlas object
convertedAtlas = BrainGlobeAtlas(atlas)
convertedAtlas.initialize(None, None)

In [ ]:
# Puts it in the scijava ObjectService for automatic discovery
ij.object().addObject(convertedAtlas)

In [ ]:
# Starts ABBA

# .. but before : logger, please shut up
DebugTools = jimport('loci.common.DebugTools')
DebugTools.enableLogging('INFO')

# Ok, let's start ABBA and its BDV view (it's also possible to start it without any GUI, 
# or even to build another GUI with a Napari view, why not ?)

ABBABdvStartCommand = jimport('ch.epfl.biop.atlas.aligner.gui.bdv.ABBABdvStartCommand') # Command import
ij.command().run(ABBABdvStartCommand, True) # Starts it